# Playground to implement W&B as well as start hyperparameter-tuning

# WanDB

In [1]:
#%pip install wandb

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c34807ad2b4c9a776b8f5db2cd5e4addb3fd01c47196600bc9c68b7ab28e9d63
  Stored in directory: c:\users\micro\appdata\local\pip\cache\wheels\ea\b7\8b\84e94095ea418b9442f5abeba4ca7b0ad52d3fe7b69d6238a6
Successfully built pathtools
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\micro\OneDrive\Dokumente\GitHub\BachelorThesis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import subprocess
import wandb
from wandb.keras import WandbCallback

In [5]:
subprocess.call(['wandb', 'login', '4e8d3dcb1584ad129b3b49ccc34f65b20116ae54'])

0

In [18]:
wandb.init(project='precursor_charge_prediction')

## Setup

In [9]:
#%pip install seaborn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\micro\OneDrive\Dokumente\GitHub\BachelorThesis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [69]:
import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import wandb
from wandb.keras import WandbCallback

# Niklas Dataset batches + split

In [73]:
import re
# the dictionary
aa_syntax_dictionary = dict()
for index, i in enumerate(list('XACDEFGHIKLMNPQRSTVWY')): # added X for 0 value
    aa_syntax_dictionary[i] = index * 100
    if i != 'X': # ignore 0 value for X
        for count in range(0, 100):
            aa_syntax_dictionary[i + "[UNIMOD:" + str(count) + "]"] = index * 100 + count

def seq_translator(sequence, dictionary=aa_syntax_dictionary):
    """
    Translates a sequence into a vector of integers
    :param sequence: string
    :param dictionary: dictionary
    :return: list
    """
    pattern = r'[A-Z]\[[^\]]*\]|.' # regex pattern to match amino acids and modifications

    result = [match for match in re.findall(pattern, sequence)]


    #print(result)
    # Fill the list with "X" characters until it reaches a length of 40
    result += ['X'] * (40 - len(result))

    return [dictionary[aa] for aa in result]


In [61]:
print(seq_translator('AAC[UNIMOD:4]LLVAW'))

['A', 'A', 'C[UNIMOD:4]', 'L', 'L', 'V', 'A', 'W']
[100, 100, 204, 1000, 1000, 1800, 100, 1900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [70]:
file_list = ["data/"+file for file in os.listdir('data') if file.endswith('.parquet')]
print(file_list[0])

data/Thermo_SRM_Pool_meta_data.parquet


In [74]:
batches_parquet = dict()
for file in file_list:
    df = pd.read_parquet(file, engine='fastparquet')
    # drop all columns we dont need for training
    for column in df.columns:
        if column not in ["modified_sequence", "precursor_charge"]:
            df.drop(column, axis=1, inplace=True)
    df["modified_sequence"] = df["modified_sequence"].apply(seq_translator)
    batches_parquet[file] = df
    # TODO dublicates ?? should be safe because of copy. please validate !

In [75]:
print(batches_parquet[file_list[0]].shape)

(6080606, 2)


In [76]:
batches_parquet[file_list[0]].head()

,modified_sequence,precursor_charge
0,"[1000, 1300, 600, 1600, 1000, 400, 1700, 2000,...",2
1,"[700, 600, 1600, 1000, 1400, 400, 2000, 1000, ...",2
2,"[1800, 400, 400, 400, 400, 400, 800, 1200, 160...",2
3,"[1000, 1300, 600, 1600, 1000, 400, 1700, 2000,...",2
4,"[1200, 1600, 1600, 1700, 100, 400, 800, 1200, ...",2


In [78]:
batches_parquet[file_list[0]].iloc[:].to_numpy().reshape(32, 2, 2)

ValueError: cannot reshape array of size 12161212 into shape (32,2,2)

### import preprocessed df's

In [12]:
cce_df = pd.read_csv('testdata/cce.csv')
# scce_df = pd.read_csv('testdata/ssce.csv')
# multilable_df = pd.read_csv('testdata/multilable.csv') ## WIP

### SCCE / CCE Model

dataset-split

In [27]:
# Split the data into train, validation, and test sets
X_2 = np.array(cce_df['sequence_vector'].tolist())
y_2 = np.array(cce_df['precursor_charge'])
max_len = max(cce_df.loc[:, 'sequence_vector'].apply(len))  # Find the maximum length


# Create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform the split
train_val_indices, test_indices = next(sss.split(X_2, y_2))
X_2_train_val, X_2_test = X_2[train_val_indices], X_2[test_indices]
y_2_train_val, y_2_test = y_2[train_val_indices], y_2[test_indices]

# Create another instance of StratifiedShuffleSplit for train-validation split
sss_train_val = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform the train-validation split
train_indices, val_indices = next(sss_train_val.split(X_2_train_val, y_2_train_val))
X_2_train, X_2_val = X_2_train_val[train_indices], X_2_train_val[val_indices]
y_2_train, y_2_val = y_2_train_val[train_indices], y_2_train_val[val_indices]

num_classes = 8  # Number of precursor charge classes (1 to 7, plus an extra class for 'None' charge)
y_2_train_encoded = tf.keras.utils.to_categorical(y_2_train, num_classes)
y_2_val_encoded = tf.keras.utils.to_categorical(y_2_val, num_classes)
y_2_test_encoded = tf.keras.utils.to_categorical(y_2_test, num_classes) # TODO war y_2_train

In [28]:
X_2.shape

(6080604,)

### KerasTuner

In [29]:
import datetime
logdir = os.path.join("logs_kerastuner", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback_kerastuner = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [31]:
#%pip install keras-tuner
import keras_tuner
from tensorflow import keras

def build_model(hp):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_len, output_dim=20, input_length=X_2.shape[0]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=hp.Int('dense_1_units', min_value=16, max_value=256, step=8),
                          activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    #hp.Choice('unknown', values=[1, 10])
    ])
    # Compile the model
    learning_rate = hp.Float("lr", min_value=0.00001, max_value=0.001, sampling="log"),
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    overwrite=True,
    executions_per_trial=3,
    directory='output_tuner',)

tuner.search(train_ds, epochs=30, validation_data=val_ds, callbacks=[tensorboard_callback_kerastuner, wandb_callback])

best_model = tuner.get_best_models()[0]

NameError: name 'train_ds' is not defined

In [ ]:
from tensorboard import program

tracking_address = "logs_kerastuner" # the path of your log file.

if __name__ == "__main__":
    tb = program.TensorBoard()
    tb.configure(argv=[None, '--logdir', tracking_address])
    url = tb.launch()
    print(f"Tensorflow listening on {url}")

### Model

In [15]:
# Define model
model_cce = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_len, output_dim=20, input_length=X_2.shape[1]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model_cce.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint('precursor_charge_prediction_model_v1/cce_wo7_allSequences.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

IndexError: tuple index out of range

In [14]:
# Train the model
history_cce = model_cce.fit(X_2_train, y_2_train_encoded, epochs=10, batch_size=32, validation_data=(X_2_val, y_2_val_encoded), callbacks=[checkpoint_callback, early_stopping, wandb_callback]) #, wandb_callback])

NameError: name 'model_cce' is not defined

In [ ]:
# Access the loss, validation loss, and accuracy from the history object
loss = history_cce.history['loss']
val_loss = history_cce.history['val_loss']
accuracy = history_cce.history['accuracy']
val_accuracy = history_cce.history['val_accuracy']

# Plot the loss, validation loss, and accuracy curves
epochs = range(1, len(loss) + 1)

# Create subplots
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot loss and validation loss
ax1.plot(epochs, loss, 'b', label='Training Loss')
ax1.plot(epochs, val_loss, 'r', label='Validation Loss')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()

# Plot accuracy and validation accuracy
ax2.plot(epochs, accuracy, 'b', label='Training Accuracy')
ax2.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()

# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

## Check in with Franzi's group for reporting

### Multilable Model
#### WIP in precursor_charge_predictor

## Model Testing
### check if models only predict charge 2 or also other charges. Due to 'overrepresentation' the best bet for the model could be to only output charge state 2

## Hyperparameter Tuning